In [1]:
import pandas as pd
import numpy as np

In [2]:
# autoregression = pd.read_csv("../outputs/predictions/sequifier-default-best-1575-all-autoregression-predictions.csv")
onestep = pd.read_csv("../outputs/predictions/sequifier-default4-best-100-predictions.csv")
onestep.shape

(658, 9)

In [3]:
onestep

,sequenceId,Australia,British,Canada,Switzerland,China,Japan,NewZealand,Singapore
0,1,0.833002,1.489911,0.820057,0.981445,0.150731,0.008502,0.727128,0.727008
1,1,0.833156,1.489557,0.820275,0.981414,0.150715,0.008498,0.727396,0.726991
2,1,0.832765,1.489219,0.819940,0.981312,0.150691,0.008496,0.727123,0.726757
3,1,0.831641,1.492283,0.818621,0.978990,0.150454,0.008462,0.726650,0.726126
4,1,0.830642,1.487950,0.818438,0.980785,0.150611,0.008483,0.725267,0.725662
...,...,...,...,...,...,...,...,...,...
653,1,0.873667,1.408342,0.851124,1.055347,0.157023,0.009645,0.748162,0.736495
654,1,0.872622,1.408136,0.850393,1.054303,0.156999,0.009636,0.747398,0.736095
655,1,0.873446,1.407739,0.851290,1.054382,0.157019,0.009643,0.748084,0.736376
656,1,0.874515,1.407176,0.851594,1.056528,0.156940,0.009676,0.749172,0.736908


In [4]:
target = pd.read_csv("../data/exchange_rate.txt").drop(columns=["itemPosition"])

In [5]:
target2 = pd.read_parquet("../data/exchange_rate-split2.parquet")["target"].values.reshape(onestep.shape[0], 8)
import json
with open("../configs/ddconfigs/exchange_rate.json", "r") as f:
    min_max_values = json.loads(f.read())["min_max_values"]

In [6]:
def invert_normalization(min_max_values, values, target_column):
    min_ = min_max_values[target_column]["min"]
    max_ = min_max_values[target_column]["max"]
    return np.array(
        [(((v + 0.8) / 1.6) * (max_ - min_)) + min_ for v in values.flatten()]
    ).reshape(*values.shape)

In [7]:
min_max_values

{'Australia': {'min': 0.483297, 'max': 1.102536},
 'British': {'min': 1.211534, 'max': 2.109},
 'Canada': {'min': 0.618582, 'max': 1.091524},
 'Switzerland': {'min': 0.548617, 'max': 1.374079},
 'China': {'min': 0.109292, 'max': 0.237954},
 'Japan': {'min': 0.006254, 'max': 0.013202},
 'NewZealand': {'min': 0.393153, 'max': 0.882379},
 'Singapore': {'min': 0.523834, 'max': 0.832556}}

In [8]:
target2["target"].values.reshape(onestep.shape[0], 8)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [9]:
columns = ["Australia", "British", "Canada", "Switzerland", "China", "Japan", "NewZealand", "Singapore"]

In [10]:
#np.mean(np.abs(autoregression[columns].values - target[columns].tail(738).values))

In [11]:
np.mean(np.abs( (np.tile(target[columns].tail(659).head(1).values, 738)).reshape(738, len(columns)) - target[columns].tail(738).values))

0.03041429962737127

In [12]:
np.mean(np.abs(onestep[columns].values - target[columns].tail(658).values))

0.03525030574756994

In [13]:
np.mean(np.abs(target[columns].tail(659).head(658).values - target[columns].tail(658).values))

0.002476956496960486